In [1]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

In [2]:
train = pd.read_csv('./train.csv', encoding = 'utf-8-sig')
test = pd.read_csv('./test.csv', encoding = 'utf-8-sig')

In [3]:
samples = []

for i in range(10):
    sample = f"input : {train['input'][i]} \n output : {train['output'][i]}"
    samples.append(sample)

In [5]:
FINETUNE_MODEL = "./finetune_weight_1"
model = AutoModelForCausalLM.from_pretrained(FINETUNE_MODEL, device_map= {"":0})

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [6]:
model_id = 'beomi/gemma-ko-7b'

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

In [7]:
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer
)

Device set to use cuda:0


In [8]:
def query_to_prompt(query):
  messages = [
      {
          "role": "system",
          "content": (
              "You are a helpful assistant specializing in restoring obfuscated Korean reviews. "
              "Your task is to transform the given obfuscated Korean review into a clear, correct, "
              "and natural-sounding Korean review that reflects its original meaning. "
              "Below are examples of obfuscated Korean reviews and their restored forms:\n\n"
              f"Example, {samples}"
              "Spacing and word length in the output must be restored to the same as in the input. "
              "Do not provide any description. Print only in Korean."
          )
      },
      {
          "role": "user",
          "content": f"input : {query}, output : "
      },
  ]

  prompt = "\n".join([m["content"] for m in messages]).strip()

  return prompt

In [23]:
def restore_review(query):
  prompt = query_to_prompt(query)
  query_len = len(query)

  outputs = pipe(
      prompt,
      do_sample=True,
      temperature=0.2,
      top_p=0.9,
      max_new_tokens=len(query),
      eos_token_id=pipe.tokenizer.eos_token_id
  )

  generated_text = outputs[0]['generated_text']
  result = generated_text[len(prompt):].strip()

  # clean
  result = result.split("'")[0]
  result = result.split("\n")[0]
  if 'output' in result:
    result = result[:query_len]

  return result

In [18]:
query = test['input'][3]
prompt = query_to_prompt(query)
print(prompt)

You are a helpful assistant specializing in restoring obfuscated Korean reviews. Your task is to transform the given obfuscated Korean review into a clear, correct, and natural-sounding Korean review that reflects its original meaning. Below are examples of obfuscated Korean reviews and their restored forms:

Example, ['input : 별 한 게토 았깝땀. 왜 싸람듯릭 펼 1캐를 쥰눈징 컥꺾폰 싸람믐롯섞 맒록 섧멍핥쟈닐 탯끎룐눈 녀뮤 퀼교... 야뭍툰 둠 변 닺씨 깍낄 싫훈 굣. 깸삥읊 20여 년 댜녁뵨 곧 중 쩨윌 귑푼 낙팠떤 곶. \n output : 별 한 개도 아깝다. 왜 사람들이 별 1개를 주는지 겪어본 사람으로서 말로 설명하자니 댓글로는 너무 길고... 아무튼 두 번 다시 가길 싫은 곳. 캠핑을 20여 년 다녀본 곳 중 제일 기분 나빴던 곳.', 'input : 잚많 쟉꼬 갉 태 좋눼욥. 차못동 줆 ㅋ \n output : 잠만 자고 갈 때 좋네요. 잠옷도 줌 ㅋ', 'input : 절테 간면 않 된는 굣 멥몫 \n output : 절대 가면 안 되는 곳 메모', 'input : 야... 칵컥 좋꾜 부됴 뼝 뚫렷썹 신원햐쥠만 닮패 넴센 밌쪄벅림. 샥퀘 핥류만 묵겠댜! 한눈 쌀람한뗌많 쭈쳔. 탐패 냄쌕갊 묘둔 쟝졈울 까저갼눈 콧. 놂랙팡엣셔 칵좋 닮패왕 윳흥예 천렸욹 택 냐눈 넴쌘갸 꼐쏙 방웨 잊슴 ㅆ... 샨닉깎 할 맑 엽숨. \n output : 아... 가격 좋고 뷰도 뻥 뚫려서 시원하지만 담배 냄새 미쳐버림. 싸게 하루만 묵겠다! 하는 사람한테만 추천. 담배 냄새가 모든 장점을 가져가는 곳. 노래방에서 각종 담배와 유흥에 쩔었을 때 나는 냄새가 계속 방에 있음 ㅆ... 싸니까 할 말 없음.', '

In [24]:
result = restore_review(query)
print(result)

뷰 맛집~~ 그런데 방음이 미흡하네요. 층간 소음과 발코니가 이중창이 아니라서 밤에 파도 소리, 아침에 칼마귀인지 카마귀인지 계속 울어서 잠을 못 잤어요ㅠ 그런데 뷰는 너무 좋아용~~~


In [25]:
restored_reviews = []

for index, row in test.iterrows():
    print(f"index: {index}")

    query = row['input']
    query_len = len(query)

    result = restore_review(query)

    restored_reviews.append(result)

index: 0
index: 1
index: 2
index: 3
index: 4


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


index: 5
index: 6
index: 7
index: 8
index: 9
index: 10
index: 11
index: 12
index: 13
index: 14
index: 15
index: 16
index: 17
index: 18
index: 19
index: 20
index: 21
index: 22
index: 23
index: 24
index: 25
index: 26
index: 27
index: 28
index: 29
index: 30
index: 31
index: 32
index: 33
index: 34
index: 35
index: 36
index: 37
index: 38
index: 39
index: 40
index: 41
index: 42
index: 43
index: 44
index: 45
index: 46
index: 47
index: 48
index: 49
index: 50
index: 51
index: 52
index: 53
index: 54
index: 55
index: 56
index: 57
index: 58
index: 59
index: 60
index: 61
index: 62
index: 63
index: 64
index: 65
index: 66
index: 67
index: 68
index: 69
index: 70
index: 71
index: 72
index: 73
index: 74
index: 75
index: 76
index: 77
index: 78
index: 79
index: 80
index: 81
index: 82
index: 83
index: 84
index: 85
index: 86
index: 87
index: 88
index: 89
index: 90
index: 91
index: 92
index: 93
index: 94
index: 95
index: 96
index: 97
index: 98
index: 99
index: 100
index: 101
index: 102
index: 103
index: 104


In [26]:
restored_reviews

['너무너무 만족스러운 호텔이에요. 부산에 오면 꼭 추천하고 싶은 곳이네요. 최고입니다! ㅎㅎ 다음에 또 올 것 같아요.',
 '프론트가 없고, 조식도 없으며, 일반 입주민들이 사용하는 사우나임에 있어 호텔처럼 관리가 잘 안 되는 느낌이네요. 가성비가 떨어져요.',
 '진짜 불친절해요. 살면서 머물렀던 호텔 중에 최악이었습니다. 직원인지 사장인지 체크인할 때부터 친절함 1도 없었구요. 3인 예약하고 추가금액까지 결제했는데 숙연 추가료 요청드리니 큰정상 수건 4개밖에 체크인 안 된대요. 2시간 정도 여유하고 다시 들어갈 때 몇 호신냐, 2분 예약하지 않았냐 물어보셨어요(체크인할 때와 같은 분). 추가금 결제했고 갔을 때 결제 내역 물어봤는데도 깸빵효 몇 번인지 결제한 시간, 분, 추가질 물어보시더라구요. 결제한 영수증 찾아 찍어 놓으셨구요. 엎이가 없고 상당히 불쾌했습니다. 그리고 냉장고에서 생선 비린내 나요. 물에서도 비린맛 났어요…, ',
 '뷰 맛집~~ 그런데 방음이 미흡하네요. 층간 소음과 발코니가 이중창이 아니라서 밤에 파도 소리, 아침에 칼마귀인지 카마귀인지 계속 울어서 잠을 못 잤어요ㅠ 그런데 뷰는 너무 좋아용~~~',
 '방 상태는 진짜 폐허 지정인데 전망은 좋아요. 보일러가 아주 찬잔하게 돌아서 추웠어요. 화장실엔 비누만 있어서 샴푸 같은 거 다 갖고 와야 해요.',
 '광안대교 뷰 하나만 보고 갔는데, 객실도 넓고 나름 깨끗했습니다! 제가 키가 안 좋아서 체크인할 때 우븐 힙밴이 있었는데, 직원분이 친절하게 대해주셨습니다! 주차를 할 수 있다는 점은 좋았지만, 주차 관리인이 불친절해서 아쉽습니다.',
 '광안대교가 잘 보여 숙소에서 편안하게 뷰를 감상할 수 있어서 좋았습니다. 객실이 전체적으로 화이트 톤이라서 사진을 찍으면 더 화사하게 나오고, 입구에서부터 신발을 벗고 들어가서 더욱더 청결한 느낌을 받았습니다. 화장실, 샤워실, 세면대가 각각 분리되어 있는 것도 인상 깊었습니다. 직원분들도 친절하시고, 웰컴 키트, 조식 쿠폰 키트도 감사합니다!

In [27]:
submission = pd.read_csv('./sample_submission.csv', encoding = 'utf-8-sig')

In [28]:
submission['output'] = restored_reviews

In [29]:
submission.to_csv('./submission.csv', index = False, encoding = 'utf-8-sig')